In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from multiprocessing import Process
from empath import Empath
import pandas as pd
import numpy as np
import textblob
import spacy
import time
import csv
import re

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stopWords = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')
prog = re.compile("(@[A-Za-z0-9]+)|([^0-9A-Za-z' \t])|(\w+:\/\/\S+)")
prog2 = re.compile(" +")
lexicon = Empath()
empath_cols = ["{0}_empath".format(v) for v in lexicon.cats.keys()]

In [3]:
def lemmatization(x, nlp):
    tweets = " ".join(list(x.values))
    letters_only = prog.sub(" ", tweets)
    lemmatized = []
    for token1 in nlp(letters_only):
        if token1.lemma_ != "-PRON-" and token1 not in stopWords:
            lemmatized.append(token1.lemma_)
        else:
            lemmatized.append(token1.text)
    final = prog2.sub(" ", " ".join(lemmatized))
    return final


def empath_analysis(x):
    val = lexicon.analyze(x, normalize=True)
    if val is None:
        return lexicon.analyze(x)
    else:
        return val
#print(lemmatization(x,nlp))

In [4]:
def processing(vals, columns, iterv):
    users = pd.DataFrame(vals)
    users = users[columns]

    print("{0}-------------".format(iterv))

    # PRE-PROCESSING

    users_text = users.groupby(["uid"])["tweet"].apply(lambda x: lemmatization(x, nlp)).reset_index()
    print("{0}-------------PRE-PROCESSING".format(iterv))

    # SENTIMENT ANALYSIS

    sentiment_arr = np.array(list(users_text["tweet"].apply(lambda x: textblob.TextBlob(str(x)).sentiment).values))
    sentiment_cols = ["sentiment", "subjectivity"]
    df_sentiment = pd.DataFrame(sentiment_arr, columns=sentiment_cols, index=users_text.uid.values)
    print("{0}-------------SENTIMENT".format(iterv))

    # EMPATH ANALYSIS

    lexicon_arr = np.array(list(users_text["tweet"].apply(lambda x: empath_analysis(x)).values))
    df_empath = pd.DataFrame.from_records(index=users_text.uid.values, data=lexicon_arr)
    df_empath.columns = empath_cols
    print("{0}-------------EMPATH".format(iterv))

    # MERGE TO SINGLE

    df = pd.DataFrame(pd.concat([df_empath, df_sentiment], axis=1))
    #df.set_index("uid", inplace=True)
    df.to_csv("G:\\sufe\\mental illness\\Dataset\\labeled\\negative\\data\\feature\\users_content_{0}.csv".format(iterv))
    print("-------------{0}".format(iterv))
#print(data_df.columns.values.tolist())

In [6]:
import csv
N=54
filepath='G:\\sufe\\mental illness\\Dataset\\labeled\\negative\\data\\'
cols = ['favorite_count', 'favorited', 'id', 'in_reply_to_status_id', 'lang', 'retweet_count', 'retweeted', 'time', 'tweet', 'uid']
for i in range(N):
    filename = filepath + 'negative_tweet{0}'.format(i)+'.csv'
    #data_df = pd.read_csv(filename)
    #data_df.columns=['favorite_count', 'favorited', 'id', 'in_reply_to_status_id', 'lang', 'retweet_count', 'retweeted', 'time', 'tweet', 'uid']
    #data_df.to_csv(filename,index=False)
    f =open(filename,'r',encoding='utf-8')
    csv_dict_reader = csv.DictReader(f)
    v=[]
    for line in csv_dict_reader:
        #提取特征
        v.append(line)
    s = time.time()
    processing(v, cols, i)
    print(time.time() - s)        
        

    

0-------------
0-------------PRE-PROCESSING
0-------------SENTIMENT
0-------------EMPATH
-------------0
88.48106098175049
1-------------
1-------------PRE-PROCESSING
1-------------SENTIMENT
1-------------EMPATH
-------------1
140.73204946517944
2-------------
2-------------PRE-PROCESSING
2-------------SENTIMENT
2-------------EMPATH
-------------2
104.23796224594116
3-------------
3-------------PRE-PROCESSING
3-------------SENTIMENT
3-------------EMPATH
-------------3
180.86334490776062
4-------------
4-------------PRE-PROCESSING
4-------------SENTIMENT
4-------------EMPATH
-------------4
134.61669969558716
5-------------
5-------------PRE-PROCESSING
5-------------SENTIMENT
5-------------EMPATH
-------------5
327.53973412513733
6-------------
6-------------PRE-PROCESSING
6-------------SENTIMENT
6-------------EMPATH
-------------6
185.84062910079956
7-------------
7-------------PRE-PROCESSING
7-------------SENTIMENT
7-------------EMPATH
-------------7
615.8762259483337
8-------------
8--